In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract the image link

def get_image_link(soup):
    
    try:
        image = soup.find('img',attrs= {'class':"a-dynamic-image"})
        if image is not None:
            image_link = image['src']
        else:
            image_link = ""
    
    except AttributeError:
        image_link = ""
    
    return image_link


# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find('span',attrs= {'class':'a-price a-text-price a-size-medium apexPriceToPay'}).find('span',attrs= {'class':'a-offscreen'}).text.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price


# Function to extract Product Rating
def get_review(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating


# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

In [69]:
if __name__ == '__main__':

    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15',
        'Accept-Language': 'en-US,en;q=0.5'
    }

    # The webpage URL
    URL = "https://www.amazon.com/s?k=pharmacy&crid=2VCG3VT3OEWYO&sprefix=pharmacy%2Caps%2C350&ref=nb_sb_ss_ts-doa-p_2_8"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[],"image-link": [],"price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        if not link.startswith('http'):
            link = "https://www.amazon.com" + link
        new_webpage = requests.get(link, headers=HEADERS)
    
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
    
        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['image-link'].append(get_image_link(new_soup))  # Add this line

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv('Scraped_datasets/Pharmacy.csv', header=True, index=False)


In [70]:
amazon_df

,title,image-link,price,rating,reviews,availability
1,Farmacy Honey Halo Ceramide Face Moisturizer C...,https://m.media-amazon.com/images/I/41Nj++2b2T...,$48.00,4.6 out of 5 stars,"2,713 ratings",In Stock
2,Farmacy Natural Cleansing Balm - Green Clean M...,https://m.media-amazon.com/images/I/41lQ4dEI-q...,$65.00,4.7 out of 5 stars,"7,296 ratings",In Stock
3,Farmacy Brightening Toner for Face - Toner wit...,https://m.media-amazon.com/images/I/51DBzCUMhs...,,4.3 out of 5 stars,155 ratings,Not Available
4,Farmacy Daily Greens Oil Free Gel Face Moistur...,https://m.media-amazon.com/images/I/41Y0nuiFrs...,$44.00,4.5 out of 5 stars,913 ratings,In Stock
5,Farmacy Whipped Greens Face Wash - Oil Free Fo...,https://m.media-amazon.com/images/I/31VByWNiK9...,$29.00,4.7 out of 5 stars,952 ratings,In Stock
...,...,...,...,...,...,...
61,"Focus Factor Adults Extra Strength, 120 Count ...",https://m.media-amazon.com/images/I/71d5-piMN1...,,4.2 out of 5 stars,"9,006 ratings",In Stock
62,PCA SKIN Hydrating Face Toner - Alcohol-Free M...,https://m.media-amazon.com/images/I/31Ndv9rLT0...,,4.6 out of 5 stars,747 ratings,In Stock
63,"PCA SKIN Nutrient Face Toner, Pumpkin Wine-Bas...",https://m.media-amazon.com/images/I/31D5Tnt3rF...,,4.6 out of 5 stars,"1,687 ratings",In Stock
64,PTCB Exam Study Cards: PTCB Exam Prep 2024-202...,https://m.media-amazon.com/images/I/31CbBY39UI...,,4.5 out of 5 stars,522 ratings,In Stock


In [71]:
df = pd.read_csv('Scraped_datasets/Pharmacy.csv')
df

,title,image-link,price,rating,reviews,availability
0,Farmacy Honey Halo Ceramide Face Moisturizer C...,https://m.media-amazon.com/images/I/41Nj++2b2T...,$48.00,4.6 out of 5 stars,"2,713 ratings",In Stock
1,Farmacy Natural Cleansing Balm - Green Clean M...,https://m.media-amazon.com/images/I/41lQ4dEI-q...,$65.00,4.7 out of 5 stars,"7,296 ratings",In Stock
2,Farmacy Brightening Toner for Face - Toner wit...,https://m.media-amazon.com/images/I/51DBzCUMhs...,NaN,4.3 out of 5 stars,155 ratings,Not Available
3,Farmacy Daily Greens Oil Free Gel Face Moistur...,https://m.media-amazon.com/images/I/41Y0nuiFrs...,$44.00,4.5 out of 5 stars,913 ratings,In Stock
4,Farmacy Whipped Greens Face Wash - Oil Free Fo...,https://m.media-amazon.com/images/I/31VByWNiK9...,$29.00,4.7 out of 5 stars,952 ratings,In Stock
...,...,...,...,...,...,...
60,"Focus Factor Adults Extra Strength, 120 Count ...",https://m.media-amazon.com/images/I/71d5-piMN1...,NaN,4.2 out of 5 stars,"9,006 ratings",In Stock
61,PCA SKIN Hydrating Face Toner - Alcohol-Free M...,https://m.media-amazon.com/images/I/31Ndv9rLT0...,NaN,4.6 out of 5 stars,747 ratings,In Stock
62,"PCA SKIN Nutrient Face Toner, Pumpkin Wine-Bas...",https://m.media-amazon.com/images/I/31D5Tnt3rF...,NaN,4.6 out of 5 stars,"1,687 ratings",In Stock
63,PTCB Exam Study Cards: PTCB Exam Prep 2024-202...,https://m.media-amazon.com/images/I/31CbBY39UI...,NaN,4.5 out of 5 stars,522 ratings,In Stock
